# Sentiment Analysis using the New York Times API

In [1]:
# Import statements
import csv
import pandas as pd
import numpy as np
import time
%matplotlib inline

In [2]:
# Using API tool with key
from nytimesarticle import articleAPI
api = articleAPI('6030f83cb2a273755c7c7145f6b91168:18:73659797')

In [3]:
# Read in the dates to use for sentiment analysis
dframe=pd.read_csv('RFE/data/IYZ_from_2010-01-04_2015-12-02.csv')
date_list = list(dframe['date'])
cleaned_dates = []
dates_dict = {}
for entry in date_list:
    cleaned_dates.append(entry.replace('-', ''))
    dates_dict[entry.replace('-', '')] = 0.5
print len(dates_dict)

1490


In [4]:
# This function takes in a response to the NYT api and parses the articles into a list of dictionaries
def parse_articles(articles, datestamp):  
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['date'] = datestamp
        dic['text'] = i['headline']['main'].encode("utf8")
        if i['snippet'] is not None:
            dic['text'] = dic['text'] + " " + i['snippet'].encode("utf8")     
        news.append(dic)
    return(news) 

In [5]:
# This function accepts a list of dates and returns a dictionary of parsed articles for those dates 
def get_articles(dates,query):
    all_articles = []
    for date in dates:
        articles = api.search(q = query,
                fq = {'news_desk':['Business','Financial','Outlook','Personal Investing','Wealth']},
                begin_date = date,
                end_date = date,
                sort='oldest')
        articles = parse_articles(articles,date)
        if len(articles) != 0:
            all_articles  = all_articles + articles
        time.sleep(0.1)
    return(all_articles)

In [6]:
# Telecommunications_articles
tele_articles = get_articles(cleaned_dates,'economy')
tele_df = pd.DataFrame.from_dict(tele_articles)

In [7]:
# Store dataframe in a CSV for future analysis
tele_df.to_csv('sentiment_data/economy.csv')

In [ ]:
# Read the CSV after manual classification of text as positive or negative
tele_df = pd.read_csv('sentiment_data/tele.csv')

In [ ]:
tele_df

In [ ]:
#From: http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6010
# AFINN is a list of English words rated for valence with an integer
# between minus five (negative) and plus five (positive). The words have
# been manually labeled by Finn Årup Nielsen in 2009-2011.
afinnfile = open("AFINN/AFINN-111.txt")
scores = {}
for line in afinnfile:
    term, score = line.split("\t")
    scores[term] = int(score)
scores

In [ ]:
def calc_sent(text):
    score = 0
    for word in text.split(" "):
        if word in scores:
            score += scores.get(word,0)
    #rescale score to 0-1 range
    return (score/10.0)+0.5

In [ ]:
print calc_sent("Firms Selling Apps for Simple Phones Software companies want to sell functions similar to those found on the iPhone to users of much simpler phones.")
print calc_sent("Nuance Communications Buys SpinVox, Which Converts Voice Mail to Text SpinVox was sold to Nuance Communications for $102.5 million, far less than the company was thought to be worth in recent years.")
print calc_sent("AT&T to Sell Smartphones Using the Android System The wireless carrier will offer five new devices by makers including Dell, Motorola and HTC.")
print calc_sent("Chinese Looking in America, but Not Buying As the United States housing market continues to struggle, potential buyers from China are checking out the market in increasing numbers.")
print calc_sent("Under Low-Key Chief, Canal Plus Prospers The French cable television group has defied skeptics, reversed the damage from Vivendi's failed expansion and insulated itself against the advertising recession.")

In [ ]:
tele_df.rename(columns={'Unnamed: 0':'sentiment'}, inplace=True)
tele_df['sentiment'] = tele_df['text'].apply(calc_sent)
tele_df

In [ ]:
bydate = tele_df.groupby('date')
temp_df = bydate.aggregate(np.mean).reset_index()
temp_df

In [ ]:
for index, row in temp_df.iterrows():
    dates_dict[str(int(row['date']))] = row['sentiment']
print len(dates_dict)

In [ ]:
dates_dict

In [ ]:
sent_df = pd.DataFrame(dates_dict.items(), columns=['date', 'sentiment']).sort('date')
print sent_df.shape
sent_df

In [ ]:
# Store dataframe in a CSV for prediction
sent_df.to_csv('data/tele_sent.csv')